In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with open("./apikey", 'r') as f:
    api_key=f.readline()[:-1]
    print(len(api_key))
    pass



DOS = Density of States


In [ ]:
from mp_api.client import MPRester
material_id="mp-12627"
# The api key is saved in a file named "apikey"
with MPRester(api_key) as mpr:
    structure = mpr.get_structure_by_material_id(material_id)
    bs = mpr.get_bandstructure_by_material_id(material_id)
    dos = mpr.get_dos_by_material_id(material_id)
    # Search by material ID or formula
    summary = mpr.summary.search(material_ids=material_id)


    # ph_bandstructure = mpr.get_phonon_bandstructure_by_material_id("mp-561")
    # ph_dos = mpr.get_phonon_dos_by_material_id("mp-561")
    pass

In [ ]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

# Analyze symmetry
sga = SpacegroupAnalyzer(structure)
crystal_system = sga.get_crystal_system()
spacegroup = sga.get_space_group_symbol()
number = sga.get_space_group_number()



print(f"Crystal system: {crystal_system}")
print(f"Space group: {spacegroup} (No. {number})")

# Extract name-related fields
print("Pretty Formula:", summary[0].formula_pretty    )
print("Full Formula:", summary[0].formula_pretty)
print("Chemical System:", summary[0].chemsys)
print("Material ID:", summary[0].material_id)

In [ ]:
# help(bs)

# Bandstructure and DOS using API

In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.electronic_structure.plotter import BSPlotter, DosPlotter, BSDOSPlotter

In [ ]:
plotter=BSPlotter(bs)
plotter.get_plot()
# plotter.plot_brillouin()

In [ ]:
plotter.plot_brillouin()

In [ ]:
plotter_dos = DosPlotter(sigma=0.01)
plotter_dos.add_dos("Total dos", dos)
plotter_dos.get_plot()


plt.xlim(-3,3)
plt.ylim(0,40)

In [ ]:
bsdos_plotter=BSDOSPlotter(bs_projection='elements', dos_projection='elements')
bsdos_plotter.get_plot(bs, dos=dos)

# Using raw data

### Bandstructure

Indexing

branch   : A particular direction in k-space

branches : Contains information about 'start' and 'end' index of all branches. 

kpoints  : List of 3D vectors in k-space. Index range specified in branches for a particular branch corresponds to particular direction or band.

bands    : an array with indexing [band_index, kpoint_index]

projections (dict[Spin, NDArray]): Orbital projections as {spin: array}.

 |              The indices of the array are [band_index, kpoint_index, orbital_index, ion_index].
 |               If the band structure is not spin polarized, we only store one data set under Spin.up.
 

In [ ]:
data=bs.as_dict()
print(data.keys())
bands=np.array(data['bands']['1'])
projections=np.array(data['projections']['1'])
kpoints=np.array(data['kpoints'])
e_fermi = data['efermi']

In [ ]:
data['branches']



In [ ]:
def kpoint_distances_for_branch(kpoints, branch, branch_index):
    the_branch=branch[branch_index]
    thelist=kpoints[the_branch['start_index']: the_branch['end_index']+1]
    thelist=list(map(np.linalg.norm, thelist))
    return np.array(thelist)

def get_band_for_branch(bands, branch, branch_index):
    the_branch=branch[branch_index]
    thelist = np.array(bands)
    thelist=thelist[:,the_branch['start_index']: the_branch['end_index']+1]
    return thelist.T


print(kpoint_distances_for_branch(data['kpoints'], data['branches'], 7).shape)
print(get_band_for_branch(data['bands']['1'], data['branches'], 7).shape)

In [ ]:
branch_index = 0


print(data['branches'][branch_index])
x = kpoint_distances_for_branch(data['kpoints'], data['branches'], branch_index)
y = get_band_for_branch(data['bands']['1'], data['branches'], branch_index) - e_fermi
# print(x)
# y = bands[:,0:37].T
plt.figure(figsize=(1,4), dpi=200)
print(y.shape)
plt.plot(x, y)
plt.ylim(-1,1)
plt.axhline(y=0, color='k')
plt.ylabel("E-E_f")
plt.xlabel(r"${}$".format(data['branches'][branch_index]['name']))



### Filtering dispersion curves in given range. Used for bands near fermi level and to export it in csv file.

In [ ]:
import pandas as pd

In [ ]:
def get_bands_smaller_than_E(max_energy=3):
    chosen_band_index = []
    for i in range(y.shape[1]):
        if(np.max(np.abs(y[:,i])) < max_energy):
            chosen_band_index.append(i)
            pass
    print(chosen_band_index)
    print(len(chosen_band_index))
    return chosen_band_index



In [ ]:
branch_index = 8
print(data['branches'][branch_index])
x = kpoint_distances_for_branch(data['kpoints'], data['branches'], branch_index)
y = get_band_for_branch(data['bands']['1'], data['branches'], branch_index) - e_fermi
# print(x)
# y = bands[:,0:37].T
plt.figure(figsize=(1,4), dpi=200)
print(x.shape)
print(y.shape)

chosen_band_index = get_bands_smaller_than_E(max_energy=3)
plt.plot(x, y[:,chosen_band_index])
# plt.ylim(-1,1)
plt.axhline(y=0, color='k')
plt.ylabel("E-E_f")
plt.xlabel(r"${}$".format(data['branches'][branch_index]['name']))

In [ ]:
df = pd.DataFrame(np.c_[x, y[:,chosen_band_index]])
df.to_csv("temp.csv", index=False, sep="\t")

## Different format to export to a file

scaled k, 3D k-vectors (in 1/Å), 3D k-vectors (in pi/lattice_const.), energy of selected bands

In [ ]:
a, b, c = bs.structure.lattice.abc

mult_latt_by_pi = lambda kk: [kk[0]*a/np.pi, kk[1]*b/np.pi, kk[2]*c/np.pi]

In [ ]:


# Get reciprocal lattice
rec_latt = bs.structure.lattice.reciprocal_lattice

# Get the k-points along the path in fractional coords
frac_kpoints = [kp.frac_coords for kp in bs.kpoints]

# Convert to Cartesian k-vectors (in 1/Å)
cart_kpoints = [rec_latt.get_cartesian_coords(kf) for kf in frac_kpoints]

# in unit of pi/lattice_constant
kpoints_pi_by_a_unit = [mult_latt_by_pi(kk) for kk in cart_kpoints]



In [ ]:
# Energy range in eV
emin, emax = -3.0, 3.0  # relative to Fermi level

# Get band energies
bands = bs.bands  # {Spin.up: [[E1_k1, E1_k2, ...], [E2_k1, ...]], ...}
keys = list(bands.keys())
if len(keys) != 1:
    print("More than 1 spin")
    pass

print(list(bands.keys()))
spin = keys[0]  # Usually Spin.up for nonmagnetic.

print(bs.efermi)
energies = [[e - bs.efermi for e in band] for band in bands[spin]]

# Find bands that have any energy within the range
selected_band_indices = []
for i, band in enumerate(energies):
    if any(emin <= e <= emax for e in band):
        selected_band_indices.append(i)

print(f"Bands in {emin} to {emax} eV range (relative to VBM):")
print(selected_band_indices)
print("total band count ", len(selected_band_indices))

In [ ]:
np.savetxt("band_index.txt", np.array(selected_band_indices).reshape(1,-1), fmt="%d", delimiter='\t')

In [ ]:
energies = np.array(energies)
energies.shape

In [ ]:
energies[selected_band_indices,].shape

In [ ]:
energies[selected_band_indices,]

In [ ]:
all_k = np.c_[np.array(frac_kpoints), np.array(cart_kpoints), np.array(kpoints_pi_by_a_unit), energies[selected_band_indices,].T]
all_k

In [ ]:
np.savetxt("temp3.txt", all_k, delimiter='\t', fmt='%10.8f')

## Density of States using raw data

In [ ]:
dos_data = dos.as_dict()

print(type(dos_data['energies']))
print(dos_data['efermi'])

x = np.array(dos_data['energies']) - dos_data['efermi']
y = dos_data['densities']['1']

plt.figure(dpi=200)

plt.plot(x, y)
plt.axvline(x=0, color='k')
plt.xlabel("E-E_f")
plt.ylabel("densities")
plt.xlim(-3,3)
plt.ylim(0,40)